In [1]:
import constraint

class Lab:
    def __init__(self, lab_number, capacity):
        self.lab_number = lab_number
        self.capacity = capacity
        self.schedule = []
        self.days_available = []

# Define a function to read and process the file
def process_schedule_file(file_path):
    schedule_data = {}
    with open(file_path) as f:
        for line in f:
            line_data = line.strip().split(',')
            time_slot = line_data[0]
            days = line_data[1]
            course_code = line_data[2]
            course_name = line_data[3]
            credits = int(line_data[4])
            instructor = line_data[5]
            enrolled_students = int(line_data[6])
            max_capacity = int(line_data[7])
            schedule_data[course_code] = {
                'TimeSlot': time_slot,
                'Days': days,
                'CourseName': course_name,
                'Instructor': instructor,
                'EnrolledStudents': enrolled_students,
                'MaxCapacity': max_capacity
            }
    return schedule_data

file_path = 'SofSUndergraduteCourseSchedule.txt'
schedule_data = process_schedule_file(file_path)

labs = [
 Lab(1, 60), Lab(2, 60), Lab(3, 60), Lab(4, 60), Lab(5, 60),
 Lab(6, 60), Lab(7, 60), Lab('GLab', 100), Lab('HLab', 100), Lab('SLab', 100),
]

#Grouping the courses according to time
def group_by_time(schedule_data):
 grouped_schedule = {}
 for course_code, entry in schedule_data.items():
     time_slot = entry["TimeSlot"]
     days = entry["Days"]
     key = f"{time_slot}/{days}"
     if key in grouped_schedule:
         grouped_schedule[key][course_code] = entry
     else:
         grouped_schedule[key] = {course_code: entry}
 return grouped_schedule

grouped_schedule = group_by_time(schedule_data)


def assign_labs(grouped_schedule, labs):
 problem = constraint.Problem()

 for course_code, _ in grouped_schedule.items():
   problem.addVariable(course_code, [lab.lab_number for lab in labs])

 # Add constraint to ensure each lab is assigned at most once
 problem.addConstraint(constraint.AllDifferentConstraint())

 solution = problem.getSolution()
 return solution

# assign_labs(grouped_schedule, labs)

for key, courses in grouped_schedule.items():
    print(f"Time Slot and Days: {key}")
    print("| {:<15} | {:<10} | {:<20} | {:<18} | {:<30} |".format("Course Code", "Lab Number", "Instructor", "Enrolled Students", "Status"))
    print("-" * 96)  # Separator line
    lab_assignment_table = "| Course Code | Lab Number | Instructor | Enrolled Students | Status |\n"
    lab_assignment_table += "| --- | --- | --- | --- | --- |\n"
    for course_code, details in courses.items():
        allocated_lab = None
        for lab in labs:
            if details['TimeSlot'] in lab.schedule and details['Days'] in lab.days_available:
                continue  # Lab already occupied at this time and day
            if len(lab.schedule) < lab.capacity:
                if details['Days'] not in lab.days_available:
                    lab.days_available.append(details['Days'])  # Update lab's available days
                allocated_lab = lab
                break
        if allocated_lab:
            allocated_lab.schedule.append(details['TimeSlot'])
            status = f"Assigned to Lab {allocated_lab.lab_number}"
        else:
            status = "Unable to assign due to capacity"
        lab_assignment_table += f"| {course_code} | {allocated_lab.lab_number if allocated_lab else '-'} | {details['Instructor']} | {details['EnrolledStudents']} | {status} |\n"
        print("| {:<15} | {:<10} | {:<20} | {:<18} | {:<30} |".format(course_code, allocated_lab.lab_number if allocated_lab else '-', details['Instructor'], details['EnrolledStudents'], status))
     

    for lab in labs:
     lab.days_available = []

Time Slot and Days: 7:00-8:40AM/-M-W---
| Course Code     | Lab Number | Instructor           | Enrolled Students  | Status                         |
------------------------------------------------------------------------------------------------
| APT1030         | 1          | Tocho.J              | 32                 | Assigned to Lab 1              |
| APT2010         | 2          | Kihara.T             | 34                 | Assigned to Lab 2              |
| APT3050         | 3          | Staff                | 39                 | Assigned to Lab 3              |
| APT3080         | 4          | Ogore.F              | 24                 | Assigned to Lab 4              |
| IST1020         | 5          | Staff                | 85                 | Assigned to Lab 5              |
| IST4035         | 6          | Sifuna.A             | 48                 | Assigned to Lab 6              |
Time Slot and Days: 9:00-10:40AM/-M-W---
| Course Code     | Lab Number | Instructor         